# Pandas

The `numpy` module is excellent for numerical computations, but to handle missing data or arrays with mixed types takes more work. The `pandas` module is currently the most widely used tool for data manipulation, providing high-performance, easy-to-use data structures and advanced data analysis tools.

In particular `pandas` features:

* A fast and efficient "DataFrame" object for data manipulation with integrated indexing;
* Tools for reading and writing data between in-memory data structures and different formats (CSV, Excel, SQL, HDF5);
* Intelligent data alignment and integrated handling of missing data;
* Intelligent label-based slicing, fancy indexing, and subsetting of large data sets;
* Aggregating or transforming data with a powerful "group-by" engine; 
* High performance merging and joining of data sets;
* Hierarchical axis indexing provides an intuitive way of working with high-dimensional data in a lower-dimensional data structure;
* Time series-functionalities;
* Highly optimized for performance, with critical code paths written in Cython or C.


In [1]:
import pandas as pd
import numpy as np

## Series

Series are completely equivalent to 1D array but with axis labels and the possibility to store heterogeneous elements. Of paramount importance are the time-series, used to define time evolutions of a phenomenon. 


In [2]:
from string import ascii_lowercase as letters

# Creating a series, accessing indexes, values and values by their index 
xs = pd.Series(np.arange(10)*0.5, index=tuple(letters[:10]))
print (xs,'\n')
print (xs.index,'\n')
# Values of the Series are actually a numpy array
print (xs.values, type(xs.values),'\n')
print (xs['f'], xs.f, xs.h, '\n')
print (xs[['d', 'f', 'h']], '\n')

a    0.0
b    0.5
c    1.0
d    1.5
e    2.0
f    2.5
g    3.0
h    3.5
i    4.0
j    4.5
dtype: float64 

Index(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j'], dtype='object') 

[0.  0.5 1.  1.5 2.  2.5 3.  3.5 4.  4.5] <class 'numpy.ndarray'> 

2.5 2.5 3.5 

d    1.5
f    2.5
h    3.5
dtype: float64 



In [3]:
# Extracting elements and operations: same as numpy array
print (xs[:3],'\n')
print (xs[7:], '\n')
print (xs[::3], '\n')
print (xs[xs>3], '\n')
print (np.exp(xs), '\n')
print (np.mean(xs), np.std(xs), '\n')

a    0.0
b    0.5
c    1.0
dtype: float64 

h    3.5
i    4.0
j    4.5
dtype: float64 

a    0.0
d    1.5
g    3.0
j    4.5
dtype: float64 

h    3.5
i    4.0
j    4.5
dtype: float64 

a     1.000000
b     1.648721
c     2.718282
d     4.481689
e     7.389056
f    12.182494
g    20.085537
h    33.115452
i    54.598150
j    90.017131
dtype: float64 

2.25 1.4361406616345072 



In [4]:
# Series can be created from python dictionary too.
# Not that the elements can be whatever!
d = {'b' : 1, 'a' : 'cat', 'c' : [2,3]}
pd.Series(d)



b         1
a       cat
c    [2, 3]
dtype: object

A key difference between Series and ndarray is that operations between Series automatically align the data based on label. Thus, you can write computations without giving consideration to whether the Series involved have the same labels.

In [5]:
s = pd.Series(np.random.randn(5), index=tuple(letters[:5]))
s[1:] + s[:-1]

a         NaN
b   -0.023920
c   -2.620759
d   -1.440774
e         NaN
dtype: float64

### Time series

Time series are very often used to profile the behaviour of a quantity as a function of time. Pandas as a special index for that, `DatetimeIndex`, that can be created e.g. with the function `pd.data_range()`

In [6]:
# to define a date, the datetime module is very useful
import datetime as dt
date = dt.date.today()
date = dt.datetime(2018,11,19,14,45,10,15)
print (date)

# otherwise, several notations are interpreted too
date = 'Nov 19 2018'
date = '19/11/2018 14:45:00'
print (date)

days = pd.date_range(date, periods=7, freq='D')
print (days)

seconds = pd.date_range(date, periods=3600, freq='s')
print (seconds)


2018-11-19 14:45:10.000015
19/11/2018 14:45:00
DatetimeIndex(['2018-11-19 14:45:00', '2018-11-20 14:45:00',
               '2018-11-21 14:45:00', '2018-11-22 14:45:00',
               '2018-11-23 14:45:00', '2018-11-24 14:45:00',
               '2018-11-25 14:45:00'],
              dtype='datetime64[ns]', freq='D')
DatetimeIndex(['2018-11-19 14:45:00', '2018-11-19 14:45:01',
               '2018-11-19 14:45:02', '2018-11-19 14:45:03',
               '2018-11-19 14:45:04', '2018-11-19 14:45:05',
               '2018-11-19 14:45:06', '2018-11-19 14:45:07',
               '2018-11-19 14:45:08', '2018-11-19 14:45:09',
               ...
               '2018-11-19 15:44:50', '2018-11-19 15:44:51',
               '2018-11-19 15:44:52', '2018-11-19 15:44:53',
               '2018-11-19 15:44:54', '2018-11-19 15:44:55',
               '2018-11-19 15:44:56', '2018-11-19 15:44:57',
               '2018-11-19 15:44:58', '2018-11-19 15:44:59'],
              dtype='datetime64[ns]', length=3600, fr

To learn more about the frequency strings, please see this [link](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases)


Timestamped data is the most basic type of time series data that associates values with points in time. For pandas objects it means using the points in time.

functions like `pd.to_datetime` can be used, for instance, when reading information as string from a dataset

In [7]:
tstamp = pd.Timestamp(dt.datetime(2018, 11, 19))

# internally it counts the nanoseconds from January 1st 19
#tstamp = pd.Timestamp(dt.datetime(1970, 1, 1, 0, 0, 0, 1))
print(tstamp.value)

# when creating a timestamp the format can be explicitly passed
print (pd.to_datetime('2010/11/12', format='%Y/%m/%d'))
print (pd.to_datetime('12-11-2010 00:00', format='%d-%m-%Y %H:%M'))


1542585600000000000
2010-11-12 00:00:00
2010-11-12 00:00:00


A standard series can be created and (range of) elements can be used as indexes

In [8]:
tseries = pd.Series(np.random.normal(10, 1, len(days)), index=days)
# Extracting elements
print (tseries[0:4], '\n')
print (tseries['2018-11-19':'2018-11-21'], '\n') # Note - includes end time


2018-11-19 14:45:00     8.644073
2018-11-20 14:45:00     9.255052
2018-11-21 14:45:00    10.131130
2018-11-22 14:45:00     9.367504
Freq: D, dtype: float64 

2018-11-19 14:45:00     8.644073
2018-11-20 14:45:00     9.255052
2018-11-21 14:45:00    10.131130
Freq: D, dtype: float64 



`pd.to_datetime` can also be used to create a `DatetimeIndex`:

In [9]:
pd.to_datetime([1, 2, 3], unit='D', origin=pd.Timestamp('1980-02-03'))

DatetimeIndex(['1980-02-04', '1980-02-05', '1980-02-06'], dtype='datetime64[ns]', freq=None)

## DataFrame

A pandas DataFrame is like a simple tabular spreadsheet. For future reference (or for people already familiar with R), a pandas DataFrame is very similar to the R DataFrame.

Each column in a DataFrame is a Series object.

The element can be whatever, missing data are dealt with too (as NaN)

### DataFrame creation

A DataFrame can be created implicitly, with, e.g., a DatatimeIndex object as index:

In [10]:
entries=10
dates=pd.date_range('19/11/2018 14:45:00',freq='h', periods=entries)
df = pd.DataFrame(np.random.randn(entries,4), index=dates, columns=['A','B','C','D'])
df


,A,B,C,D
2018-11-19 14:45:00,-1.196206,-0.005023,-1.045394,-2.510783
2018-11-19 15:45:00,-0.976083,1.285383,-1.154749,0.162501
2018-11-19 16:45:00,-0.547681,1.199528,-0.759876,-0.342517
2018-11-19 17:45:00,0.448813,-1.462292,2.184132,-0.327731
2018-11-19 18:45:00,2.764921,0.643460,0.412177,-0.101926
2018-11-19 19:45:00,0.720379,0.809215,-0.950164,-2.362155
2018-11-19 20:45:00,-0.284566,1.251222,0.153906,0.547070
2018-11-19 21:45:00,0.309338,0.498926,-0.244230,0.087305
2018-11-19 22:45:00,-0.942123,2.091599,0.636584,0.505689
2018-11-19 23:45:00,1.054986,-0.691885,1.345434,-0.872294


or by means of a dictionary:


In [11]:
df2 = pd.DataFrame(
    { 'A' : 1.,
      'B' : pd.Timestamp('20130102'),
      'C' : pd.Series(1,index=range(4),dtype='float32'),
      'D' : np.arange(7,11),
      'E' : pd.Categorical(["test","train","test","train"]),
    }
    )
   

In [12]:
df2

,A,B,C,D,E
0,1.0,2013-01-02,1.0,7,test
1,1.0,2013-01-02,1.0,8,train
2,1.0,2013-01-02,1.0,9,test
3,1.0,2013-01-02,1.0,10,train


### Viewing Data

In [13]:
df.head()

,A,B,C,D
2018-11-19 14:45:00,-1.196206,-0.005023,-1.045394,-2.510783
2018-11-19 15:45:00,-0.976083,1.285383,-1.154749,0.162501
2018-11-19 16:45:00,-0.547681,1.199528,-0.759876,-0.342517
2018-11-19 17:45:00,0.448813,-1.462292,2.184132,-0.327731
2018-11-19 18:45:00,2.764921,0.643460,0.412177,-0.101926


In [14]:
df.tail(4)

,A,B,C,D
2018-11-19 20:45:00,-0.284566,1.251222,0.153906,0.547070
2018-11-19 21:45:00,0.309338,0.498926,-0.244230,0.087305
2018-11-19 22:45:00,-0.942123,2.091599,0.636584,0.505689
2018-11-19 23:45:00,1.054986,-0.691885,1.345434,-0.872294


In [15]:
df.index

DatetimeIndex(['2018-11-19 14:45:00', '2018-11-19 15:45:00',
               '2018-11-19 16:45:00', '2018-11-19 17:45:00',
               '2018-11-19 18:45:00', '2018-11-19 19:45:00',
               '2018-11-19 20:45:00', '2018-11-19 21:45:00',
               '2018-11-19 22:45:00', '2018-11-19 23:45:00'],
              dtype='datetime64[ns]', freq='H')

In [16]:
df.columns

Index(['A', 'B', 'C', 'D'], dtype='object')

In [17]:
df.values

array([[-1.19620631, -0.00502277, -1.04539356, -2.51078258],
       [-0.97608321,  1.28538316, -1.15474936,  0.16250111],
       [-0.54768068,  1.19952781, -0.75987646, -0.3425171 ],
       [ 0.44881272, -1.46229245,  2.18413175, -0.32773115],
       [ 2.764921  ,  0.64345951,  0.41217707, -0.10192583],
       [ 0.7203787 ,  0.80921504, -0.95016443, -2.36215528],
       [-0.28456631,  1.25122198,  0.15390586,  0.54706999],
       [ 0.30933796,  0.49892616, -0.24423024,  0.087305  ],
       [-0.94212303,  2.09159948,  0.63658419,  0.50568934],
       [ 1.05498562, -0.69188494,  1.34543396, -0.87229372]])

In [18]:
df.describe()

,A,B,C,D
count,10.000000,10.000000,10.000000,10.000000
mean,0.135178,0.562013,0.057782,-0.521484
std,1.203850,1.043906,1.111167,1.092876
min,-1.196206,-1.462292,-1.154749,-2.510783
25%,-0.843512,0.120964,-0.902592,-0.739850
50%,0.012386,0.726337,-0.045162,-0.214828
75%,0.652487,1.238298,0.580482,0.143702
max,2.764921,2.091599,2.184132,0.547070


In [19]:
df.T

,2018-11-19 14:45:00,2018-11-19 15:45:00,2018-11-19 16:45:00,2018-11-19 17:45:00,2018-11-19 18:45:00,2018-11-19 19:45:00,2018-11-19 20:45:00,2018-11-19 21:45:00,2018-11-19 22:45:00,2018-11-19 23:45:00
A,-1.196206,-0.976083,-0.547681,0.448813,2.764921,0.720379,-0.284566,0.309338,-0.942123,1.054986
B,-0.005023,1.285383,1.199528,-1.462292,0.643460,0.809215,1.251222,0.498926,2.091599,-0.691885
C,-1.045394,-1.154749,-0.759876,2.184132,0.412177,-0.950164,0.153906,-0.244230,0.636584,1.345434
D,-2.510783,0.162501,-0.342517,-0.327731,-0.101926,-2.362155,0.547070,0.087305,0.505689,-0.872294


In [20]:
df.sort_index(axis=1,ascending=False)

,D,C,B,A
2018-11-19 14:45:00,-2.510783,-1.045394,-0.005023,-1.196206
2018-11-19 15:45:00,0.162501,-1.154749,1.285383,-0.976083
2018-11-19 16:45:00,-0.342517,-0.759876,1.199528,-0.547681
2018-11-19 17:45:00,-0.327731,2.184132,-1.462292,0.448813
2018-11-19 18:45:00,-0.101926,0.412177,0.643460,2.764921
2018-11-19 19:45:00,-2.362155,-0.950164,0.809215,0.720379
2018-11-19 20:45:00,0.547070,0.153906,1.251222,-0.284566
2018-11-19 21:45:00,0.087305,-0.244230,0.498926,0.309338
2018-11-19 22:45:00,0.505689,0.636584,2.091599,-0.942123
2018-11-19 23:45:00,-0.872294,1.345434,-0.691885,1.054986


In [21]:
df.sort_values(by="C")

,A,B,C,D
2018-11-19 15:45:00,-0.976083,1.285383,-1.154749,0.162501
2018-11-19 14:45:00,-1.196206,-0.005023,-1.045394,-2.510783
2018-11-19 19:45:00,0.720379,0.809215,-0.950164,-2.362155
2018-11-19 16:45:00,-0.547681,1.199528,-0.759876,-0.342517
2018-11-19 21:45:00,0.309338,0.498926,-0.244230,0.087305
2018-11-19 20:45:00,-0.284566,1.251222,0.153906,0.547070
2018-11-19 18:45:00,2.764921,0.643460,0.412177,-0.101926
2018-11-19 22:45:00,-0.942123,2.091599,0.636584,0.505689
2018-11-19 23:45:00,1.054986,-0.691885,1.345434,-0.872294
2018-11-19 17:45:00,0.448813,-1.462292,2.184132,-0.327731


## Selection

### Getting slices

The following show how to get part of the DataFrame (i.e. not just the elements)

In [22]:
## standard and safe
print (df['A'],'\n')

## equivalent but dangerous (imagine blank spaces in the name of the column..)
print (df.A)

2018-11-19 14:45:00   -1.196206
2018-11-19 15:45:00   -0.976083
2018-11-19 16:45:00   -0.547681
2018-11-19 17:45:00    0.448813
2018-11-19 18:45:00    2.764921
2018-11-19 19:45:00    0.720379
2018-11-19 20:45:00   -0.284566
2018-11-19 21:45:00    0.309338
2018-11-19 22:45:00   -0.942123
2018-11-19 23:45:00    1.054986
Freq: H, Name: A, dtype: float64 

2018-11-19 14:45:00   -1.196206
2018-11-19 15:45:00   -0.976083
2018-11-19 16:45:00   -0.547681
2018-11-19 17:45:00    0.448813
2018-11-19 18:45:00    2.764921
2018-11-19 19:45:00    0.720379
2018-11-19 20:45:00   -0.284566
2018-11-19 21:45:00    0.309338
2018-11-19 22:45:00   -0.942123
2018-11-19 23:45:00    1.054986
Freq: H, Name: A, dtype: float64


In [23]:
# selecting rows by counting
print (df[0:3])

# or by index
print (df["2018-11-19 14:45:00":"2018-11-19 16:45:00"])

                            A         B         C         D
2018-11-19 14:45:00 -1.196206 -0.005023 -1.045394 -2.510783
2018-11-19 15:45:00 -0.976083  1.285383 -1.154749  0.162501
2018-11-19 16:45:00 -0.547681  1.199528 -0.759876 -0.342517
                            A         B         C         D
2018-11-19 14:45:00 -1.196206 -0.005023 -1.045394 -2.510783
2018-11-19 15:45:00 -0.976083  1.285383 -1.154749  0.162501
2018-11-19 16:45:00 -0.547681  1.199528 -0.759876 -0.342517


### Selection by label

In [26]:
# getting a cross section (part of the DataFrame) using a label
df.loc[dates[0]]

A   -1.196206
B   -0.005023
C   -1.045394
D   -2.510783
Name: 2018-11-19 14:45:00, dtype: float64

In [27]:
# selecting on a multi-axis by label:
df.loc[:,['A','B']]

,A,B
2018-11-19 14:45:00,-1.196206,-0.005023
2018-11-19 15:45:00,-0.976083,1.285383
2018-11-19 16:45:00,-0.547681,1.199528
2018-11-19 17:45:00,0.448813,-1.462292
2018-11-19 18:45:00,2.764921,0.643460
2018-11-19 19:45:00,0.720379,0.809215
2018-11-19 20:45:00,-0.284566,1.251222
2018-11-19 21:45:00,0.309338,0.498926
2018-11-19 22:45:00,-0.942123,2.091599
2018-11-19 23:45:00,1.054986,-0.691885


In [28]:
# showing label slicing, both endpoints are included:
df.loc['2018-11-19 18:45:00':'2018-11-19 20:45:00',['A','B']]

,A,B
2018-11-19 18:45:00,2.764921,0.643460
2018-11-19 19:45:00,0.720379,0.809215
2018-11-19 20:45:00,-0.284566,1.251222


In [29]:
# getting an individual element
print (df.loc[dates[1],'A'])

# equivalently
print (df.at[dates[1],'A'])

-0.9760832072505768
-0.9760832072505768


### Selecting by position

In [30]:
# select via the position of the passed integers:
print (df.iloc[3],'\n')

# notation similar to numpy/python
print (df.iloc[3:5,0:2])

A    0.448813
B   -1.462292
C    2.184132
D   -0.327731
Name: 2018-11-19 17:45:00, dtype: float64 

                            A         B
2018-11-19 17:45:00  0.448813 -1.462292
2018-11-19 18:45:00  2.764921  0.643460


In [31]:
# selecting raws 1,2 and 4 for columns 0 and 2
df.iloc[[1,2,4],[0,2]]

,A,C
2018-11-19 15:45:00,-0.976083,-1.154749
2018-11-19 16:45:00,-0.547681,-0.759876
2018-11-19 18:45:00,2.764921,0.412177


In [32]:
# slicing rows explicitly
print (df.iloc[1:3,:],'\n')

# slicing columns explicitly
print (df.iloc[:,1:3])


                            A         B         C         D
2018-11-19 15:45:00 -0.976083  1.285383 -1.154749  0.162501
2018-11-19 16:45:00 -0.547681  1.199528 -0.759876 -0.342517 

                            B         C
2018-11-19 14:45:00 -0.005023 -1.045394
2018-11-19 15:45:00  1.285383 -1.154749
2018-11-19 16:45:00  1.199528 -0.759876
2018-11-19 17:45:00 -1.462292  2.184132
2018-11-19 18:45:00  0.643460  0.412177
2018-11-19 19:45:00  0.809215 -0.950164
2018-11-19 20:45:00  1.251222  0.153906
2018-11-19 21:45:00  0.498926 -0.244230
2018-11-19 22:45:00  2.091599  0.636584
2018-11-19 23:45:00 -0.691885  1.345434


In [33]:
# selecting an individual element by position
df.iloc[1,1]
df.iat[1,1]


1.285383159256027

### Boolean index

Very powerful way of filtering out data with certain features. Notation is very similar to numpy arrays.

In [34]:
# Filter by a boolean condition on the values of a single column
df[df['B'] > 0]

,A,B,C,D
2018-11-19 15:45:00,-0.976083,1.285383,-1.154749,0.162501
2018-11-19 16:45:00,-0.547681,1.199528,-0.759876,-0.342517
2018-11-19 18:45:00,2.764921,0.643460,0.412177,-0.101926
2018-11-19 19:45:00,0.720379,0.809215,-0.950164,-2.362155
2018-11-19 20:45:00,-0.284566,1.251222,0.153906,0.547070
2018-11-19 21:45:00,0.309338,0.498926,-0.244230,0.087305
2018-11-19 22:45:00,-0.942123,2.091599,0.636584,0.505689


In [35]:
# Selecting on the basis of boolean conditions applied to the whole DataFrame
df[df>0]

# a DataFrame with the same shape is returned, with NaN's where condition is not met

,A,B,C,D
2018-11-19 14:45:00,NaN,NaN,NaN,NaN
2018-11-19 15:45:00,NaN,1.285383,NaN,0.162501
2018-11-19 16:45:00,NaN,1.199528,NaN,NaN
2018-11-19 17:45:00,0.448813,NaN,2.184132,NaN
2018-11-19 18:45:00,2.764921,0.643460,0.412177,NaN
2018-11-19 19:45:00,0.720379,0.809215,NaN,NaN
2018-11-19 20:45:00,NaN,1.251222,0.153906,0.547070
2018-11-19 21:45:00,0.309338,0.498926,NaN,0.087305
2018-11-19 22:45:00,NaN,2.091599,0.636584,0.505689
2018-11-19 23:45:00,1.054986,NaN,1.345434,NaN


### Setting

Combination of selection and setting of values

In [36]:
# setting values by label (same as by position)
df.at[dates[0],'A'] = 0

# setting and assigning a numpy array
df.loc[:,'D'] = np.array([5] * len(df))

# defining a brend new column
df['E'] = np.arange(len(df))*0.5

# defining a brend new column by means of a pd.Series: indexes must be the same!
df['E prime'] = pd.Series(np.arange(len(df))*2, index=df.index)


In [37]:
def dcos(theta):
    theta = theta*(np.pi/180)
    return np.cos(theta)
 
df['cosine'] = pd.Series(df["E"].apply(dcos), index=df.index)
df

,A,B,C,D,E,E prime,cosine
2018-11-19 14:45:00,0.000000,-0.005023,-1.045394,5,0.0,0,1.000000
2018-11-19 15:45:00,-0.976083,1.285383,-1.154749,5,0.5,2,0.999962
2018-11-19 16:45:00,-0.547681,1.199528,-0.759876,5,1.0,4,0.999848
2018-11-19 17:45:00,0.448813,-1.462292,2.184132,5,1.5,6,0.999657
2018-11-19 18:45:00,2.764921,0.643460,0.412177,5,2.0,8,0.999391
2018-11-19 19:45:00,0.720379,0.809215,-0.950164,5,2.5,10,0.999048
2018-11-19 20:45:00,-0.284566,1.251222,0.153906,5,3.0,12,0.998630
2018-11-19 21:45:00,0.309338,0.498926,-0.244230,5,3.5,14,0.998135
2018-11-19 22:45:00,-0.942123,2.091599,0.636584,5,4.0,16,0.997564
2018-11-19 23:45:00,1.054986,-0.691885,1.345434,5,4.5,18,0.996917


In [38]:
# another example of global setting
df2=df.copy()
df2[df2>0] = -df2
df2

,A,B,C,D,E,E prime,cosine
2018-11-19 14:45:00,0.000000,-0.005023,-1.045394,-5,0.0,0,-1.000000
2018-11-19 15:45:00,-0.976083,-1.285383,-1.154749,-5,-0.5,-2,-0.999962
2018-11-19 16:45:00,-0.547681,-1.199528,-0.759876,-5,-1.0,-4,-0.999848
2018-11-19 17:45:00,-0.448813,-1.462292,-2.184132,-5,-1.5,-6,-0.999657
2018-11-19 18:45:00,-2.764921,-0.643460,-0.412177,-5,-2.0,-8,-0.999391
2018-11-19 19:45:00,-0.720379,-0.809215,-0.950164,-5,-2.5,-10,-0.999048
2018-11-19 20:45:00,-0.284566,-1.251222,-0.153906,-5,-3.0,-12,-0.998630
2018-11-19 21:45:00,-0.309338,-0.498926,-0.244230,-5,-3.5,-14,-0.998135
2018-11-19 22:45:00,-0.942123,-2.091599,-0.636584,-5,-4.0,-16,-0.997564
2018-11-19 23:45:00,-1.054986,-0.691885,-1.345434,-5,-4.5,-18,-0.996917


### Dropping

N.B.: dropping doesn't act permanently on the DataFrame, i.e. to get that do :
```python
df = df.drop(....)
```

In [39]:
# Dropping by column
df.drop(['E prime'], axis=1)

#which is equivalent to
df.drop(columns=['E prime'])

,A,B,C,D,E,cosine
2018-11-19 14:45:00,0.000000,-0.005023,-1.045394,5,0.0,1.000000
2018-11-19 15:45:00,-0.976083,1.285383,-1.154749,5,0.5,0.999962
2018-11-19 16:45:00,-0.547681,1.199528,-0.759876,5,1.0,0.999848
2018-11-19 17:45:00,0.448813,-1.462292,2.184132,5,1.5,0.999657
2018-11-19 18:45:00,2.764921,0.643460,0.412177,5,2.0,0.999391
2018-11-19 19:45:00,0.720379,0.809215,-0.950164,5,2.5,0.999048
2018-11-19 20:45:00,-0.284566,1.251222,0.153906,5,3.0,0.998630
2018-11-19 21:45:00,0.309338,0.498926,-0.244230,5,3.5,0.998135
2018-11-19 22:45:00,-0.942123,2.091599,0.636584,5,4.0,0.997564
2018-11-19 23:45:00,1.054986,-0.691885,1.345434,5,4.5,0.996917


In [40]:
# Dropping by raws
# save and always working
df.drop(df.index[[1,2,3,4]])

,A,B,C,D,E,E prime,cosine
2018-11-19 14:45:00,0.000000,-0.005023,-1.045394,5,0.0,0,1.000000
2018-11-19 19:45:00,0.720379,0.809215,-0.950164,5,2.5,10,0.999048
2018-11-19 20:45:00,-0.284566,1.251222,0.153906,5,3.0,12,0.998630
2018-11-19 21:45:00,0.309338,0.498926,-0.244230,5,3.5,14,0.998135
2018-11-19 22:45:00,-0.942123,2.091599,0.636584,5,4.0,16,0.997564
2018-11-19 23:45:00,1.054986,-0.691885,1.345434,5,4.5,18,0.996917


In [41]:
# something like df.drop('index_name') 
# would work but the type of index must be specificed, 
# in particular with DatetimeIndex
df.drop(pd.to_datetime("2018-11-19 22:45:00"))

,A,B,C,D,E,E prime,cosine
2018-11-19 14:45:00,0.000000,-0.005023,-1.045394,5,0.0,0,1.000000
2018-11-19 15:45:00,-0.976083,1.285383,-1.154749,5,0.5,2,0.999962
2018-11-19 16:45:00,-0.547681,1.199528,-0.759876,5,1.0,4,0.999848
2018-11-19 17:45:00,0.448813,-1.462292,2.184132,5,1.5,6,0.999657
2018-11-19 18:45:00,2.764921,0.643460,0.412177,5,2.0,8,0.999391
2018-11-19 19:45:00,0.720379,0.809215,-0.950164,5,2.5,10,0.999048
2018-11-19 20:45:00,-0.284566,1.251222,0.153906,5,3.0,12,0.998630
2018-11-19 21:45:00,0.309338,0.498926,-0.244230,5,3.5,14,0.998135
2018-11-19 23:45:00,1.054986,-0.691885,1.345434,5,4.5,18,0.996917


## Missing data

pandas primarily uses the value np.nan to represent missing data. It is by default not included in computations.

In [42]:
df_wNan = df[df>0]
df_wNan

,A,B,C,D,E,E prime,cosine
2018-11-19 14:45:00,NaN,NaN,NaN,5,NaN,NaN,1.000000
2018-11-19 15:45:00,NaN,1.285383,NaN,5,0.5,2.0,0.999962
2018-11-19 16:45:00,NaN,1.199528,NaN,5,1.0,4.0,0.999848
2018-11-19 17:45:00,0.448813,NaN,2.184132,5,1.5,6.0,0.999657
2018-11-19 18:45:00,2.764921,0.643460,0.412177,5,2.0,8.0,0.999391
2018-11-19 19:45:00,0.720379,0.809215,NaN,5,2.5,10.0,0.999048
2018-11-19 20:45:00,NaN,1.251222,0.153906,5,3.0,12.0,0.998630
2018-11-19 21:45:00,0.309338,0.498926,NaN,5,3.5,14.0,0.998135
2018-11-19 22:45:00,NaN,2.091599,0.636584,5,4.0,16.0,0.997564
2018-11-19 23:45:00,1.054986,NaN,1.345434,5,4.5,18.0,0.996917


In [43]:
# dropping raws with at least a Nan
df_wNan.dropna(how='any')

,A,B,C,D,E,E prime,cosine
2018-11-19 18:45:00,2.764921,0.64346,0.412177,5,2.0,8.0,0.999391


In [44]:
# getting a mask
df_wNan.isna()
#df_wNan.notna()

,A,B,C,D,E,E prime,cosine
2018-11-19 14:45:00,True,True,True,False,True,True,False
2018-11-19 15:45:00,True,False,True,False,False,False,False
2018-11-19 16:45:00,True,False,True,False,False,False,False
2018-11-19 17:45:00,False,True,False,False,False,False,False
2018-11-19 18:45:00,False,False,False,False,False,False,False
2018-11-19 19:45:00,False,False,True,False,False,False,False
2018-11-19 20:45:00,True,False,False,False,False,False,False
2018-11-19 21:45:00,False,False,True,False,False,False,False
2018-11-19 22:45:00,True,False,False,False,False,False,False
2018-11-19 23:45:00,False,True,False,False,False,False,False


In [45]:
# filling missing data
df_wNan.fillna(value=0)

,A,B,C,D,E,E prime,cosine
2018-11-19 14:45:00,0.000000,0.000000,0.000000,5,0.0,0.0,1.000000
2018-11-19 15:45:00,0.000000,1.285383,0.000000,5,0.5,2.0,0.999962
2018-11-19 16:45:00,0.000000,1.199528,0.000000,5,1.0,4.0,0.999848
2018-11-19 17:45:00,0.448813,0.000000,2.184132,5,1.5,6.0,0.999657
2018-11-19 18:45:00,2.764921,0.643460,0.412177,5,2.0,8.0,0.999391
2018-11-19 19:45:00,0.720379,0.809215,0.000000,5,2.5,10.0,0.999048
2018-11-19 20:45:00,0.000000,1.251222,0.153906,5,3.0,12.0,0.998630
2018-11-19 21:45:00,0.309338,0.498926,0.000000,5,3.5,14.0,0.998135
2018-11-19 22:45:00,0.000000,2.091599,0.636584,5,4.0,16.0,0.997564
2018-11-19 23:45:00,1.054986,0.000000,1.345434,5,4.5,18.0,0.996917


Fill gaps forward or backward by propagating non-NA values forward or backward:

In [46]:
df_wNan.fillna(method='pad')

,A,B,C,D,E,E prime,cosine
2018-11-19 14:45:00,NaN,NaN,NaN,5,NaN,NaN,1.000000
2018-11-19 15:45:00,NaN,1.285383,NaN,5,0.5,2.0,0.999962
2018-11-19 16:45:00,NaN,1.199528,NaN,5,1.0,4.0,0.999848
2018-11-19 17:45:00,0.448813,1.199528,2.184132,5,1.5,6.0,0.999657
2018-11-19 18:45:00,2.764921,0.643460,0.412177,5,2.0,8.0,0.999391
2018-11-19 19:45:00,0.720379,0.809215,0.412177,5,2.5,10.0,0.999048
2018-11-19 20:45:00,0.720379,1.251222,0.153906,5,3.0,12.0,0.998630
2018-11-19 21:45:00,0.309338,0.498926,0.153906,5,3.5,14.0,0.998135
2018-11-19 22:45:00,0.309338,2.091599,0.636584,5,4.0,16.0,0.997564
2018-11-19 23:45:00,1.054986,2.091599,1.345434,5,4.5,18.0,0.996917


## Operations

Here comes the most relevant advantage of DataFrame. Operations on columns are extremly fast, almost as fast as the actual operation between elements in a raw

In [47]:
# Some statistics (mean() just as an example)
# raws
print (df.mean(axis=0),'\n')
# columns
print (df.mean(axis=1),'\n')

A          0.254798
B          0.562013
C          0.057782
D          5.000000
E          2.250000
E prime    9.000000
cosine     0.998915
dtype: float64 

2018-11-19 14:45:00    0.707083
2018-11-19 15:45:00    1.093502
2018-11-19 16:45:00    1.555974
2018-11-19 17:45:00    2.095758
2018-11-19 18:45:00    2.831421
2018-11-19 19:45:00    2.725497
2018-11-19 20:45:00    3.159884
2018-11-19 21:45:00    3.437453
2018-11-19 22:45:00    3.969089
2018-11-19 23:45:00    4.315065
Freq: H, dtype: float64 



In [48]:
# global operations on columns
df.apply(np.cumsum)

,A,B,C,D,E,E prime,cosine
2018-11-19 14:45:00,0.000000,-0.005023,-1.045394,5,0.0,0,1.000000
2018-11-19 15:45:00,-0.976083,1.280360,-2.200143,10,0.5,2,1.999962
2018-11-19 16:45:00,-1.523764,2.479888,-2.960019,15,1.5,6,2.999810
2018-11-19 17:45:00,-1.074951,1.017596,-0.775888,20,3.0,12,3.999467
2018-11-19 18:45:00,1.689970,1.661055,-0.363711,25,5.0,20,4.998858
2018-11-19 19:45:00,2.410349,2.470270,-1.313875,30,7.5,30,5.997906
2018-11-19 20:45:00,2.125782,3.721492,-1.159969,35,10.5,42,6.996536
2018-11-19 21:45:00,2.435120,4.220418,-1.404199,40,14.0,56,7.994670
2018-11-19 22:45:00,1.492997,6.312018,-0.767615,45,18.0,72,8.992234
2018-11-19 23:45:00,2.547983,5.620133,0.577819,50,22.5,90,9.989152


In [49]:
df.apply(lambda x: x.max() - x.min())

A           3.741004
B           3.553892
C           3.338881
D           0.000000
E           4.500000
E prime    18.000000
cosine      0.003083
dtype: float64

In [50]:
# syntax is as usual similar to that of numpy arrays
df['A']+df['B']

2018-11-19 14:45:00   -0.005023
2018-11-19 15:45:00    0.309300
2018-11-19 16:45:00    0.651847
2018-11-19 17:45:00   -1.013480
2018-11-19 18:45:00    3.408381
2018-11-19 19:45:00    1.529594
2018-11-19 20:45:00    0.966656
2018-11-19 21:45:00    0.808264
2018-11-19 22:45:00    1.149476
2018-11-19 23:45:00    0.363101
Freq: H, dtype: float64

Let's play it hard and load (in memory) a (relatively) large dataset

In [51]:
file_name="/Users/mzanetti/data/LEMMA2018/DT/raw/data_000636.txt"
data=pd.read_csv(file_name)
data

FileNotFoundError: File b'/Users/mzanetti/data/LEMMA2018/DT/raw/data_000636.txt' does not exist

Let's know do some operations among (elements of) columns

In [52]:
# the one-liner killing it all
data['timens']=data['TDC_MEAS']*25/30+data['BX_COUNTER']*25

NameError: name 'data' is not defined

In [53]:
# the old slooow way
def conversion(data):
    result=[]
    for i in range(len(data)): 
        result.append(data.loc[data.index[i],'TDC_MEAS']*25/30.+data.loc[data.index[i],'BX_COUNTER']*25)
    return result

data['timens']=conversion(data)

NameError: name 'data' is not defined

In [ ]:
data

## Merge

pandas provides various facilities for easily combining together Series, DataFrame, and Panel objects with various kinds of set logic for the indexes and relational algebra functionality in the case of join / merge-type operations.

### Concat

concatenation (adding raws) is straightforward


In [ ]:
rdf = pd.DataFrame(np.random.randn(10, 4))
rdf

In [ ]:
# divide it into pieaces raw-wise
pieces = [rdf[:3], rdf[3:7], rdf[7:]]
pieces

In [ ]:
# put it back together
pd.concat(pieces)

# indexes can be ignored
#pd.concat(pieces, ignore_index=True)

# in case of dimension mismatch, Nan are added where needed

In [ ]:
# appending a single raw (as a Series)
s = rdf.iloc[3]
rdf.append(s, ignore_index=True)


### Merge/Join

SQL like operations on table can be performed on DataFrames. This is all rather sophisticated, refer to the [doc](https://pandas.pydata.org/pandas-docs/stable/merging.html#merging) for more info/examples

In [ ]:
left = pd.DataFrame({'key': ['foo', 'bar'], 'lval': [1, 2]})
right = pd.DataFrame({'key': ['foo', 'bar'], 'rval': [4, 5]})

pd.merge(left,right,on="key")

## Grouping

By “group by” we are referring to a process involving one or more of the following steps:

* Splitting the data into groups based on some criteria
* Applying a function to each group independently
* Combining the results into a data structure


In [54]:
gdf = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar',
                          'foo', 'bar', 'foo', 'foo'],
                    'B' : ['one', 'one', 'two', 'three',
                           'two', 'two', 'one', 'three'],
                    'C' : np.random.randn(8),
                    'D' : np.random.randn(8)})
gdf

,A,B,C,D
0,foo,one,-0.681907,0.370281
1,bar,one,1.828171,-0.871728
2,foo,two,-1.333924,-0.643990
3,bar,three,0.548167,-0.251928
4,foo,two,-1.460942,1.336076
5,bar,two,-1.080928,-0.719882
6,foo,one,0.641623,0.612129
7,foo,three,1.103283,-0.603466


In [55]:
# Grouping and then applying the sum() 
# function to the resulting groups (effective only where number are there).
gdf.groupby('A').sum()

,C,D
A,,
bar,1.295410,-1.843537
foo,-1.731867,1.071030


## Multi-indexing


Hierarchical / Multi-level indexing allows sophisticated data analysis on higher dimensional data. In essence, it enables you to store and manipulate data with an arbitrary number of dimensions in lower dimensional data structures like Series (1d) and DataFrame (2d).

In [ ]:
tuples = list(zip(['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'],
          ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']))
multi_index = pd.MultiIndex.from_tuples(tuples, names=['first', 'second'])
print (index,'\n')

s = pd.Series(np.random.randn(8), index=multi_index)
print (s)


In [ ]:
# it enables further features of the groupby method,
# e.g. when group-by by multiple columns
gdf.groupby(['A','B']).sum()

In [ ]:
# stack() method “compresses” a level in the DataFrame’s columns
gdf.groupby(['A','B']).sum().stack()

## Plotting

Just a preview, more on the next lab class!

In [ ]:
ts = pd.Series(np.random.randn(1000), index=pd.date_range('1/1/2000', periods=1000))
ts.cumsum().plot()

In [ ]:
pdf=pd.DataFrame(np.random.randn(1000, 4), index=ts.index,columns=['A', 'B', 'C', 'D'])
pdf = pdf.cumsum()
plt.figure(); df.plot(); plt.legend(loc='best')